# GNC Torque Calculations - Grace



In [133]:
import numpy as np

### Constants utilized throughout calculations 

- $\mu$ = Gravitational Parameter for Earth
- $M$ = Magnetic constant of the Earth

In [134]:
# Used in velocity Calculations 
mu = 3.986*10**(14) # [m^3 / s^2] Standard Gravitational Parameter
# https://en.wikipedia.org/wiki/Standard_gravitational_parameter

# Magnetic Dipole Calculations
M = 7.96e15 # [tesla m^3] Magnetic constant of the Earth

r_earth = 6378 # [km] Radius of Earth


### Helper Functions

- ```cubesat_vel``` : Determine velocity of spacecraft given orbit radius
- ```cube_dims``` : Output dimensions X,Y,Z in prettier formats
- ```moment_inertia``` : Moment of Inertia for a rectangle

![Moment Of Inertia of a Rectangular Prism](MomentOfInertia.png)
http://mechanicsmap.psu.edu/websites/centroidtables/centroids3D/centroids3D.html


In [135]:
def cubesat_vel(r_orbit):
    '''
    mu = gravitational paramter combining G and mass of earth
    a = r_orbit, or radius of the orbit, and earth's radius added
    '''
    #https://space.stackexchange.com/questions/19592/cubesat-orbit-determination 
    a = (r_earth + r_orbit)*1000
    v = np.sqrt(mu/a)
    
    return v

def cube_dims(x,y,z):
    return x,y,z

def moment_inertia(m,h,w,d):
    '''
    mass of the laminar rectangle 
    h is height of the rectangle
    w is the width of the base of the rectangle
    d is the base of rectangle    
    '''
    I_xx = (1/12)*m*(h**2+d**2)
    I_yy = (1/12)*m*(w**2+d**2)
    I_zz = (1/12)*m*(h**2+w**2)
    return I_xx,I_yy,I_zz

## Aerodynamic/Atmospheric Torque

Only experienced near the earth's surface, very short time frame but not negligible. If torques are significant enough to use large portion of our stored momentum within reaction wheels, may be worth using **Magnetorquers**.

Equation utilized:

$$T_a = 0.5\rho C_dAV^2(c_p - c_g)$$

Where:


- $T_a$ = Aerodynamic Torque
- $\rho$ = Air Density
- $C_d$ = Drag Coefficient
- $A$ = Area
- $V$ = Spacecraft Velocity
- $c_p$ = Offset Center of Pressure
- $c_g$ = Center of Gravity


In [136]:
# Aerodynamic/Atmospheric Torque

def atmos_torque(rho,c_d,A_r,v,c_pa,c_m):
    T_a = 0.5*rho*c_d*A_r*(v**2)*abs(c_pa-c_m)
    return T_a

## Magnetic Field Torque

Only experienced near large planetary bodies. This torque will be a factor for CaliPER when located near earth. 

Equation utilized:

$$T_m = D\frac {M\lambda} {R^3}$$

Where:


- $T_m$ = Magnetic Field Torque
- $D$ = Earth's Magnetic Dipole
- $M$ = Earth's Magnetic Moment
- $\lambda$ = Unitless function of the magnetic latitude (1 at latitude, 2 at poles)
- $R$ = Radius of Spacecraft from Earth's Dipole Center

*Note, $\frac {M\lambda} {R^3}$  is calculating $B$, also known as magnetic field strength*

To get more detailed information, check out this paper from NASA: https://ntrs.nasa.gov/api/citations/19690020961/downloads/19690020961.pdf

In [137]:
# Magnetic Field Torque

def mag_torque(lamb,r_orbit,D):
    
    # D is in units of [A m^2]
    R = (r_earth + r_orbit)*1000 # [m] convert to meters
    B = (M*lamb)/(R**3) # [Nm]
    
    mag_torque = D*B # [Nm]
    return mag_torque

## Gravitational Gradient Torque

Only experienced near large planetary bodies. This torque will be a factor for CaliPER when located near earth. 

Equation utilized:

$$T_g = 3\frac {\mu} {2R^3} |I_z - I_y|sin(2\theta)$$

Where:


- $T_g$ = Gravitational Gradient Torque
- $\mu$ = Earth's standard gravitational parameter
- $\theta$ = maximum deviation of spacecraft's Z axis
- $R$ = Radius of spacecraft from earth's center
- $I_z$ = Moment of inertia (Z axis) of spacecraft
- $I_y$ = Moment of inertia (Y axis) of spacecraft

In [138]:
# Gravitational Gradient Torque

def gravity_torque(r_orbit,Iz,Iy,theta):
    '''
    Iz = #moment of intertia about Z axis in kgm2
    Iy = #moment of intertia about Y axis in kgm2
    '''
    R = (r_earth + r_orbit)*1000 # [m]
    theta = (theta*np.pi)/180 # [rad]
    grav_torque = 1.5*(mu/(R**3))*abs(Iz-Iy)*np.sin(2*theta)
    
    return grav_torque

## Solar Radiation Torque

CaliPER will experience this throughout the entire mission duration, and have the most impact on the spacecraft.

Equation utilized:

$$T_p = \frac {F_s}{c}A_s(1+q)cos(\theta)(c_p - c_g)$$

Where:


- $T_p$ = Solar Radiation Torque
- $q$ = Reflectance factor of Spacecraft
- $F_s$ = Solar FLux
- $c$ = Speed of Light
- $A_s$ = Surface area of spacecraft
- $c_p$ = Offset of center of solar pressure
- $c_g$ = Center of Gravity
- $\theta$ = Angle of incidence to the sun

In [139]:
# Gravitational Gradient Torque

def solar_torque(solar_flux,A_s,q,phi,c_pa,c_m):
    '''
    #solar flux changes as func of distance
    solar_flux = 1367 #solar constant adjusted for actual distance from the sun (W/m2)
    c = 3e8 #speed of light, m/s
    A_s = #sunlit surface area, m2
    q = #refelctance, 0 for perfect absorption, 1 for perfect reflectance
    phi = #angle of incidence from the sun
    c_pa = 0 # center of pressure
    c_m = 0.07 #center of mass in z_axis
    '''
    
    c= 3e8
    phi = (phi*np.pi)/180
    solar_torque = (solar_flux/c)*A_s*(1+q)*abs(c_pa-c_m)*np.cos(phi)
    return solar_torque

# Test using values from this paper

Paper found here: https://ntrs.nasa.gov/api/citations/20110007876/downloads/20110007876.pdf

In [140]:
v_test = cubesat_vel(700)
P1atmosY = atmos_torque(A_r=(1.2*1.1), v=v_test, rho=1e-13,c_d=2,c_pa=0.05, c_m=0)
print('atmospheric torque Nm: ',P1atmosY)
test_mag = mag_torque(lamb=1.9,r_orbit=700,D=0.5)
print('magnetic torque Nm: ',test_mag)
test_grav = gravity_torque(r_orbit=700,Iz=25,Iy=50,theta=1)
print('gravitational torque Nm: ',test_grav)
test_sol = solar_torque(solar_flux=1367,A_s=(1.2*1.1),q=0.6,phi=0,c_pa=0.1,c_m=0)
print('solar torque Nm: ',test_sol)


atmospheric torque Nm:  3.7168126589432056e-07
magnetic torque Nm:  2.132578360799018e-05
gravitational torque Nm:  1.4711469342842513e-06
solar torque Nm:  9.62368e-07


## Phase 1 Calculations

For this phase, as we in close proximity to earth after it's initial launch, we will experience **all four external torques:** 
- atmospheric
- magnetic
- solar
- gravitational gradient

*In addition, it will need to detumble after detaching from the launch vehicle.*

#### Parameters:

- Orbit: 500 km
- Density: 1.70x10^-11 kg*m^3
- Drag Coefficient: 2.5 
- Center of Pressure: 0 (Assume at the Center, nothing deployed yet)
- Dimensions of the spacecraft (nothing will be deployed yet at this stage): 
    - x = 0.2m
    - y = 0.1m
    - z = 0.3m



In [141]:
r = 500 # [km] insertion orbit altitude
x_cube,y_cube,z_cube = cube_dims(x=0.2,y=0.1,z=0.3)
worst_cgx,worst_cgy,worst_cgz = cube_dims(x=0.045,y=0.02,z=0.07)

### ------------ Aerodynamic / Atmospheric Torques ------------

rho_500km = 1.70*10**(-11) # [kg m^3] http://www.braeunig.us/space/atmos.htm
# Assume worst case, high solar activity
# https://space.stackexchange.com/questions/51485/what-is-the-density-of-the-earths-atmosphere-at-an-altitude-of-four-hundred-kil

c_d = 2.5 # Typical Drag coefficient for spacecraft (ranges between 2.0-2.5)
# https://ntrs.nasa.gov/api/citations/20110007876/downloads/20110007876.pdf
# If desired, can calculate more detailed drag coefficients at:
# http://eecs.uci.edu/~swallace//papers_wallace/pdf/GCD-Moe-June1993-Refinements.pdf


v_500km = cubesat_vel(r)
P1atmosY = atmos_torque(A_r=(x_cube*z_cube), v=v_500km, rho=rho_500km,c_d=2.5,c_pa=0, c_m=worst_cgz)
P1atmosX = 0 #not facing drag in worst case
P1atmosZ = 0 #not facing drag in worst case
print('atmospheric torque: ',P1atmosY*1e3, "[mNm]")


### ------------ Magnetic Field Torques ------------

polar_lambda = 1.9 #63 deg inclination, ISS; SMAD recommended
D_orbit = 0.1 #suggested in SMAD
P1mag = mag_torque(lamb=polar_lambda,r_orbit=r,D=D_orbit)
print('magnetic torque: ',P1mag*1e3,'[mNm]')


### ------------ Gravitational Torques ------------

max_mass_cube = 12 # [kg], max of cubesat
IxxP1, IyyP1, IzzP1 = moment_inertia(m=max_mass_cube,h=y_cube,w=x_cube,d=z_cube)
P1grav = gravity_torque(r_orbit=r,Iz=IzzP1,Iy=IyyP1,theta=45)
print('gravitational torque: ',P1grav*1e3,'[mNm]')



### ------------ Solar Radiation Torques ------------

LEO_flux=1367 # [W/m2] standard near earth
sixUreflect = 0.6 # reflectance 0.6 via SMAD
P1solY = solar_torque(solar_flux=LEO_flux,A_s=(x_cube*z_cube),q=sixUreflect,phi=0,c_pa=0,c_m=worst_cgz)
P1solX = 0 # not being hit by sun in worst case
P1solZ = 0 # not being hit by sun in worst case
print('solar torque: ',P1solY*1e3,'[mNm]') 



atmospheric torque:  0.005172295725501598 [mNm]
magnetic torque:  0.004648150209226018 [mNm]
gravitational torque:  0.00014700497276361996 [mNm]
solar torque:  3.06208e-05 [mNm]


### Questions for David and Jonathan:

#### David:
- Why do you chose $\rho$ to be 7.7e-14? This is 2 orders of mag lower than what the JPL kids last year chose 4.840*10e(-11)
    - (Page 114, 4.7.1.2) >> Caliper is expected to separate from launch vehicle at an altitude of 500 km on an escape trajectory towards Mars. Currently, the exact orbital parameters are unknown. What is known is that the launch vehicle will never pass into Earth’s shadow at any point. To meet this requirement, CaliPER’s initial trajectory will be assumed to be at a high inclination with a beta-angle between 69 and 90 degrees, longitude of the ascending node of 90 degrees, and argument of periapsis of 0 degrees. These assumptions ensure CaliPER will not pass into Earth’s shadow when heading away from Earth and that the initial motion is in the direction of Earth’s velocity vector with respect to the Sun.
- Can we assume that other two axes do nothing in drag? like could there be a case where drag with all 3 axes could cause more torques than just everything in one direction?
- Magnetic torques, I saw a source with tesla = 7.96*10^15 instead of 7.8*10^15 https://www.valispace.com/how-to-iii-calculate-satellite-disturbance-torques/
- also tiny error, you have variables D and B flipped for Magnetic torques but like that gives the same answer so i guess it okie
- I haven't checked your magnetic D and lambda values, I do wanna make sure we got those right (I couldn't open SMAD)
- gravity gradient calcs: are these valid moments of inertias if we are looking at a 3d box instaed a 2D rectangle? http://mechanicsmap.psu.edu/websites/centroidtables/centroids3D/centroids3D.html



$$\theta = P_N * 11.25-5.625$$

where $$P_N =

$$\theta = (\frac{L_1}{L_2}*P_1 +(1 - \frac{L_1}{L_2})*P_2) * 10$$

- $\theta$ = Angle
- $P_1$ = Photodiode 1 location from 1-16 on linear array
- $P_2$ = Photodiode 2 location from 1-16 on linear array
- $L_1$ = Light intensity on Photodiode 1
- $L_2$ = Light intensity on Photodiode 2